## Final Demo Analysis 
 - xgb_model.sav : best model saved
 - wildfire_predictions: probabilities of wildfire occurrences for each of the given points

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from scipy.interpolate import griddata

In [4]:
import shapely
import geopandas as gpd

In [5]:
coords = json.load(open('wildfire_occurrencesv2.json'))

lat = []
long = []
occur = []
year = []


for i in coords:
    long.append(float(i.split(',')[0]))
    lat.append(float(i.split(',')[1]))

for i in coords.values():
    occur.append(i['wildfire'])
    year.append(i['year'])

lat = pd.Series(lat)
long = pd.Series(long)
occur = pd.Series(occur)
year = pd.Series(year)


df = pd.concat((lat,long,occur,year),axis=1).rename(columns={0:'lat',1:'long',2:'occur',3:'year'})


water = json.load(open('water_way_distance.json'))

lat = []
long = []
waters = []


for i in water:
    long.append(float(i.split(',')[0]))
    lat.append(float(i.split(',')[1]))

for i in water.values():
    waters.append(i['close_to_waterway'])

lat = pd.Series(lat)
long = pd.Series(long)
waters = pd.Series(waters)


df_2 = pd.concat((lat,long,waters),axis=1).rename(columns={0:'lat',1:'long',2:'close_water'})


df = df.merge(df_2, on=['lat','long'])


test = pd.read_csv('interp_result')

test = test[['lat','lon','wind_mph_interp']]

test = test.rename(columns={'lon':'long'})

w = json.load(open('woodlands.json'))

w = pd.DataFrame(np.array(w)).rename(columns={0:'lat',1:'long'})

w['woodlands'] = 1


# adjust this
df = df.round(2)
test = test.round(2)
w = w.round(2)

df_coords = set((zip(df.lat.tolist(),df.long.tolist())))



count = 0
for i in set((zip(test.lat.tolist(),test.long.tolist()))):
    if i in df_coords:
        count += 1
print(count)

df_3 = df.merge(test, on=['lat','long'],how='left')


count = 0
for i in set((zip(w.lat.tolist(),w.long.tolist()))):
    if i in df_coords:
        count += 1
print(count)


l = []
for _, row in df.iterrows():
    s = (row.lat, row.long)
    if s in set((zip(w.lat.tolist(),w.long.tolist()))):
        l.append(1)
    else:
        l.append(0)
df_3['woodlands'] = l

elevation = np.load('coord_elevations.npy')
elevation = elevation.astype(float)
elevation = elevation[np.where(elevation[:,2] > 0)[0]]
elevation = pd.DataFrame(elevation).rename(columns={0:'lat',1:'long'})
elevation = elevation.round(2)

count = 0
for i in set(zip(elevation.lat.tolist(), elevation.long.tolist())):
    if i in df_coords:
        count += 1
print(count)

df_3 = df_3.merge(elevation, on=['lat','long'],how='left')
df_3.rename(columns={2:'elevation'},inplace=True)


d1 = np.load('datamask_40.npy')
d2 = np.load('datamask_50.npy')
d = np.concatenate((d1, d2), axis=0)
d = pd.DataFrame(d).rename(columns={0:'long',1:'lat'})
d = d.round(2)

df_3 = df_3.merge(d, on=['lat','long'], how='left').rename(columns={2:'datamask'})

c1 = np.load('tree_cover_40.npy')
c2 = np.load('tree_cover_50.npy')
c = np.concatenate((c1, c2),axis=0)
c = pd.DataFrame(c).rename(columns={0:'long',1:'lat'})
c = c.round(2)

df_3 = df_3.merge(c, on=['lat','long'],how='left').rename(columns={2:'coverpct'})




precip = pd.read_csv('noaa_precip.csv')
precip = precip.iloc[np.where(precip.year < 2021)[0]]
p_sum = precip.groupby(['lat','lon']).sum().reset_index()[['lat','lon','precip_in']]

data_points = p_sum[['lat','lon']].to_numpy()
data_points[:,1] *= -1

data_values = np.squeeze(p_sum[['precip_in']].to_numpy())

coords = np.array(list(df_coords))
print(data_points.shape)
print(data_values.shape)
interp_values = griddata(data_points, data_values, coords, method='linear')
interp_result = pd.DataFrame(np.append(coords, interp_values[:,None], 1))
interp_result.columns = ['lat', 'long', 'precip_in']
interp_result = interp_result.dropna().reset_index(drop=True)

count = 0
for i in set(zip(interp_result.lat.tolist(), interp_result.long.tolist())):
    if i in df_coords:
        count += 1
print(count)

df_3 = df_3.merge(interp_result, on=['lat','long'],how='left')

df_3.drop('year',axis=1,inplace=True)

df = df_3

ca = gpd.read_file('ca-state-boundary')
ca = ca.to_crs(4326) # reproject to WGS84 lon lat

grid = gpd.GeoSeries(gpd.points_from_xy(df.long.values, df.lat.values))
ca_idx = grid[grid.within(ca.unary_union)].index
df = df.iloc[ca_idx]

8573
4642
8522
(644, 2)
(644,)
8362


In [6]:
test = df[['lat','long','wind_mph_interp']]
data_points = test[test.wind_mph_interp.notnull()][['lat','long']].to_numpy()
coords = test[df.wind_mph_interp.isnull()][['lat','long']].to_numpy()
data_values = test[test.wind_mph_interp.notnull()].wind_mph_interp.to_numpy()

interp_values = griddata(data_points, data_values, coords, method='nearest')
interp_result = pd.DataFrame(np.append(coords, interp_values[:,None], 1))
interp_result.columns = ['lat', 'long', 'wind_mph_interp']
interp_result = interp_result.dropna().reset_index(drop=True)

df = df.merge(interp_result, on=['lat','long'], how='outer')

l = []
for i, row in df.iterrows():
    if row.wind_mph_interp_x != row.wind_mph_interp_x:
        l.append(row.wind_mph_interp_y)
    else:
        l.append(row.wind_mph_interp_x)
df['wind_mph_interp'] = l
df.drop(['wind_mph_interp_x','wind_mph_interp_y'],axis=1,inplace=True)

In [7]:
test = df[['lat','long','elevation']]
data_points = test[test.elevation.notnull()][['lat','long']].to_numpy()
coords = test[df.elevation.isnull()][['lat','long']].to_numpy()
data_values = test[test.elevation.notnull()].elevation.to_numpy()

interp_values = griddata(data_points, data_values, coords, method='nearest')
interp_result = pd.DataFrame(np.append(coords, interp_values[:,None], 1))
interp_result.columns = ['lat', 'long', 'elevation']
interp_result = interp_result.dropna().reset_index(drop=True)

df = df.merge(interp_result, on=['lat','long'], how='outer')

l = []
for i, row in df.iterrows():
    if row.elevation_x != row.elevation_x:
        l.append(row.elevation_y)
    else:
        l.append(row.elevation_x)
df['elevation'] = l
df.drop(['elevation_x','elevation_y'],axis=1,inplace=True)

In [8]:
test = df[['lat','long','precip_in']]
data_points = test[test.precip_in.notnull()][['lat','long']].to_numpy()
coords = test[df.precip_in.isnull()][['lat','long']].to_numpy()
data_values = test[test.precip_in.notnull()].precip_in.to_numpy()

interp_values = griddata(data_points, data_values, coords, method='nearest')
interp_result = pd.DataFrame(np.append(coords, interp_values[:,None], 1))
interp_result.columns = ['lat', 'long', 'precip_in']
interp_result = interp_result.dropna().reset_index(drop=True)

df = df.merge(interp_result, on=['lat','long'], how='outer')


l = []
for i, row in df.iterrows():
    if row.precip_in_x != row.precip_in_x:
        l.append(row.precip_in_y)
    else:
        l.append(row.precip_in_x)
df['precip_in'] = l
df.drop(['precip_in_x','precip_in_y'],axis=1,inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8705 entries, 0 to 8704
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   lat              8705 non-null   float64
 1   long             8705 non-null   float64
 2   occur            8705 non-null   object 
 3   close_water      8705 non-null   object 
 4   woodlands        8705 non-null   int64  
 5   datamask         8705 non-null   float64
 6   coverpct         8705 non-null   float64
 7   wind_mph_interp  8705 non-null   float64
 8   elevation        8705 non-null   float64
 9   precip_in        8705 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 748.1+ KB


In [10]:
df = df.iloc[np.where(df.datamask != 2)]

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8456 entries, 4 to 8704
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   lat              8456 non-null   float64
 1   long             8456 non-null   float64
 2   occur            8456 non-null   object 
 3   close_water      8456 non-null   object 
 4   woodlands        8456 non-null   int64  
 5   datamask         8456 non-null   float64
 6   coverpct         8456 non-null   float64
 7   wind_mph_interp  8456 non-null   float64
 8   elevation        8456 non-null   float64
 9   precip_in        8456 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 726.7+ KB


In [12]:
df.precip_in = df.precip_in / 12

In [13]:
df.rename(columns={
    'wind_mph_interp':'wind_mph',
    'elevation':'elevation_ft',
    'precip_in':'sum_precip_ft'
},inplace=True)

In [14]:
df.drop('datamask',axis=1,inplace=True)

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel

from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report
from xgboost import XGBClassifier, plot_importance

/Users/asattiraju/opt/anaconda3/envs/General/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [16]:
sc = StandardScaler()

In [17]:
df

,lat,long,occur,close_water,woodlands,coverpct,wind_mph,elevation_ft,sum_precip_ft
4,38.29,-123.00,0,0,0,1.0,17.16,32.21,22.606240
5,38.29,-122.97,0,0,0,1.0,16.61,184.32,21.448076
6,38.29,-122.95,0,0,0,4.0,17.21,48.52,20.637433
7,38.29,-122.92,0,0,0,0.0,17.39,84.65,19.421467
8,38.29,-122.89,0,0,0,1.0,17.27,56.55,18.205502
...,...,...,...,...,...,...,...,...,...
8700,41.37,-122.05,0,0,1,94.0,21.49,1428.77,31.050380
8701,41.37,-122.02,0,0,1,74.0,21.48,1374.78,30.331025
8702,41.37,-122.00,0,0,1,91.0,21.47,1272.08,29.851456
8703,41.37,-121.97,0,0,1,14.0,21.45,1242.09,29.132101


In [18]:
scaled_features = df[['lat','long','wind_mph','coverpct','elevation_ft','sum_precip_ft']]

In [19]:
scaled = pd.DataFrame(sc.fit_transform(scaled_features), columns=['lat','long','wind_mph','coverpct','elevation_ft','sum_precip_ft'])

In [20]:
binary = df[['close_water','woodlands']]

In [21]:
binary = binary.astype(int).reset_index().drop('index',axis=1)

In [22]:
binary

,close_water,woodlands
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
8451,0,1
8452,0,1
8453,0,1
8454,0,1


In [23]:
X = pd.concat([binary,scaled],axis=1)

In [24]:
y = df.occur.values.astype(int)

In [25]:
X

,close_water,woodlands,lat,long,wind_mph,coverpct,elevation_ft,sum_precip_ft
0,0,0,-1.889508,-0.271771,-0.243111,-1.284718,-1.229529,0.114527
1,0,0,-1.889508,-0.216630,-0.486148,-1.284718,-0.928711,0.006477
2,0,0,-1.889508,-0.179870,-0.221017,-1.203599,-1.197274,-0.069152
3,0,0,-1.889508,-0.124730,-0.141478,-1.311758,-1.125822,-0.182594
4,0,0,-1.889508,-0.069590,-0.194504,-1.284718,-1.181394,-0.296037
...,...,...,...,...,...,...,...,...
8451,0,1,1.674407,1.474332,1.670250,1.229999,1.532358,0.902319
8452,0,1,1.674407,1.529472,1.665831,0.689200,1.425585,0.835207
8453,0,1,1.674407,1.566232,1.661412,1.148879,1.222482,0.790466
8454,0,1,1.674407,1.621372,1.652575,-0.933199,1.163173,0.723354


In [97]:
models = []

for num in [8,6,5]:
    
    # Construct XGBoost Model and RFE object with internal estimator of Decision Tree
    xgb = XGBClassifier(n_estimators=500,subsample=0.75,colsample_bytree=0.75)
    ft_selector = RFE(estimator=DecisionTreeClassifier(),n_features_to_select=num)

    # Construct a pipeline to prevent data leakage
    steps = [('ft_selector',ft_selector),('xgb',xgb)]
    pipe = Pipeline(steps=steps)

    # Parameters for GridSearch
    params = {'xgb__max_depth':[4,6,8],'xgb__learning_rate':[0.01,0.05,0.1], 'xgb__lambda':[0.75, 1, 0.25]}

    # Construct GridSearch object, pass in the pipe object
    search_xgb = GridSearchCV(pipe, params, cv=5, scoring='accuracy',n_jobs=-1)

    search_xgb.fit(X.values,y)

    models.append(search_xgb)
    
    # Print results
    print(str(num) + ", " + str(search_xgb.best_params_) + ", " + str(search_xgb.cv_results_['mean_test_score'].max()))


/Users/asattiraju/opt/anaconda3/envs/General/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/asattiraju/opt/anaconda3/envs/General/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/asattiraju/opt/anaconda3/

/Users/asattiraju/opt/anaconda3/envs/General/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/asattiraju/opt/anaconda3/envs/General/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:57:18] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:57:30] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/asattiraju/opt/anaconda3/envs/General/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[12:57:18] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:57:30] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:57:18] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:57:30] WARNING: /Users/runner/miniforge3/cond

...

/Users/asattiraju/opt/anaconda3/envs/General/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/asattiraju/opt/anaconda3/envs/General/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:08:58] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:09:17] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:09:31] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:09:50] WARNING: /Users/runner/miniforge3/cond

/Users/asattiraju/opt/anaconda3/envs/General/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:11:02] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
5, {'xgb__lambda': 1, 'xgb__learning_rate': 0.01, 'xgb__max_depth': 4}, 0.6595425231338766
[13:10:19] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:10:39] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric i

[13:10:14] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:10:28] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:10:37] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:10:51] WARNING: /Users/runner/miniforge3/cond

In [26]:
from imblearn.over_sampling import SMOTENC

In [27]:
sm = SMOTENC(random_state=42, categorical_features=[0, 1])

In [28]:
X_res, y_res = sm.fit_resample(X, y)

In [37]:
pd.Series(y_res).value_counts()

0    4982
1    4982
dtype: int64

In [29]:
xgb = XGBClassifier(n_estimators=500,subsample=0.75,colsample_bytree=0.75,learning_rate=0.01,max_depth=4, verbosity=0,)

In [40]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report

models = []

kf = KFold(n_splits=5, shuffle=True, random_state =42)
for train_index, test_index in kf.split(X_res):
    X_train, X_test = X_res.values[train_index], X_res.values[test_index]
    y_train, y_test = y_res[train_index], y_res[test_index]
    
    xgb = XGBClassifier(n_estimators=500,subsample=0.75,colsample_bytree=0.75,learning_rate=0.01,max_depth=20, verbosity=0,)
    xgb.fit(X_train, y_train)
    
    models.append(xgb)
    y_pred = xgb.predict(X_test)
    print(classification_report(y_test, y_pred))

/Users/asattiraju/opt/anaconda3/envs/General/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


              precision    recall  f1-score   support

           0       0.86      0.89      0.87      1012
           1       0.88      0.86      0.87       981

    accuracy                           0.87      1993
   macro avg       0.87      0.87      0.87      1993
weighted avg       0.87      0.87      0.87      1993



/Users/asattiraju/opt/anaconda3/envs/General/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


              precision    recall  f1-score   support

           0       0.87      0.88      0.87      1007
           1       0.87      0.87      0.87       986

    accuracy                           0.87      1993
   macro avg       0.87      0.87      0.87      1993
weighted avg       0.87      0.87      0.87      1993



/Users/asattiraju/opt/anaconda3/envs/General/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


              precision    recall  f1-score   support

           0       0.86      0.87      0.86       975
           1       0.87      0.86      0.87      1018

    accuracy                           0.86      1993
   macro avg       0.86      0.86      0.86      1993
weighted avg       0.86      0.86      0.86      1993



/Users/asattiraju/opt/anaconda3/envs/General/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


              precision    recall  f1-score   support

           0       0.87      0.87      0.87      1024
           1       0.87      0.86      0.86       969

    accuracy                           0.87      1993
   macro avg       0.87      0.87      0.87      1993
weighted avg       0.87      0.87      0.87      1993



/Users/asattiraju/opt/anaconda3/envs/General/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


              precision    recall  f1-score   support

           0       0.85      0.86      0.86       964
           1       0.87      0.86      0.87      1028

    accuracy                           0.86      1992
   macro avg       0.86      0.86      0.86      1992
weighted avg       0.86      0.86      0.86      1992



In [139]:

fig, ax = plt.subplots(1,1,figsize=(10,6),dpi=1000)
ax.bar(x=X.columns, height=models[0].feature_importances_,color=[(i/10,i/20,i/80) for i in range(len(X.columns))])
plt.xticks(rotation=45)
plt.ylabel('Feature Importance Scores')
plt.title('Feature Importances for XGBoost Model')
plt.show()

In [140]:
import pickle
best_model = models[0]

pickle.dump(best_model, open('xgb_model_v2.sav','wb'))

In [51]:
X_res

,close_water,woodlands,lat,long,wind_mph,coverpct,elevation_ft,sum_precip_ft
0,0,0,-1.889508,-0.271771,-0.243111,-1.284718,-1.229529,0.114527
1,0,0,-1.889508,-0.216630,-0.486148,-1.284718,-0.928711,0.006477
2,0,0,-1.889508,-0.179870,-0.221017,-1.203599,-1.197274,-0.069152
3,0,0,-1.889508,-0.124730,-0.141478,-1.311758,-1.125822,-0.182594
4,0,0,-1.889508,-0.069590,-0.194504,-1.284718,-1.181394,-0.296037
...,...,...,...,...,...,...,...,...
9959,0,0,-0.603435,0.157184,0.409169,0.972654,0.895325,-0.128234
9960,0,1,0.773515,0.337404,-0.591572,1.036871,1.850938,0.235312
9961,0,0,-1.300917,1.151026,-0.500898,-0.743919,-0.315827,-0.968696
9962,1,1,0.465367,-1.310473,-0.005400,1.286851,0.872631,-0.705968


In [125]:
df

,lat,long,occur,close_water,woodlands,coverpct,wind_mph,elevation_ft,sum_precip_ft
4,38.29,-123.00,0,0,0,1.0,17.16,32.21,22.606240
5,38.29,-122.97,0,0,0,1.0,16.61,184.32,21.448076
6,38.29,-122.95,0,0,0,4.0,17.21,48.52,20.637433
7,38.29,-122.92,0,0,0,0.0,17.39,84.65,19.421467
8,38.29,-122.89,0,0,0,1.0,17.27,56.55,18.205502
...,...,...,...,...,...,...,...,...,...
8700,41.37,-122.05,0,0,1,94.0,21.49,1428.77,31.050380
8701,41.37,-122.02,0,0,1,74.0,21.48,1374.78,30.331025
8702,41.37,-122.00,0,0,1,91.0,21.47,1272.08,29.851456
8703,41.37,-121.97,0,0,1,14.0,21.45,1242.09,29.132101


In [126]:
X

,close_water,woodlands,lat,long,wind_mph,coverpct,elevation_ft,sum_precip_ft
0,0,0,-1.889508,-0.271771,-0.243111,-1.284718,-1.229529,0.114527
1,0,0,-1.889508,-0.216630,-0.486148,-1.284718,-0.928711,0.006477
2,0,0,-1.889508,-0.179870,-0.221017,-1.203599,-1.197274,-0.069152
3,0,0,-1.889508,-0.124730,-0.141478,-1.311758,-1.125822,-0.182594
4,0,0,-1.889508,-0.069590,-0.194504,-1.284718,-1.181394,-0.296037
...,...,...,...,...,...,...,...,...
8451,0,1,1.674407,1.474332,1.670250,1.229999,1.532358,0.902319
8452,0,1,1.674407,1.529472,1.665831,0.689200,1.425585,0.835207
8453,0,1,1.674407,1.566232,1.661412,1.148879,1.222482,0.790466
8454,0,1,1.674407,1.621372,1.652575,-0.933199,1.163173,0.723354


In [141]:
proba = best_model.predict_proba(X)
wildfire_proba = proba[:,1]

/Users/asattiraju/opt/anaconda3/envs/General/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [142]:
final = df[['lat','long']]
final['proba'] = wildfire_proba

/var/folders/d6/wyth4gv96fd_0vch_sfby8jh0000gn/T/ipykernel_27181/29731915.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['proba'] = wildfire_proba


In [143]:
final.reset_index(inplace=True,drop=True)

In [144]:
final

,lat,long,proba
0,38.29,-123.00,0.010365
1,38.29,-122.97,0.077522
2,38.29,-122.95,0.028626
3,38.29,-122.92,0.073912
4,38.29,-122.89,0.019934
...,...,...,...
8451,41.37,-122.05,0.054026
8452,41.37,-122.02,0.084933
8453,41.37,-122.00,0.049899
8454,41.37,-121.97,0.043618


In [145]:
final.to_csv('wildfire_predictions_v2.csv')